In [1]:
import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from GameDayFunctions.fangraphs_projection_2020 import Projection
#from GameDayFunctions.draft_2020 import Draft

In [2]:
year = 2020
path_data = "projections/"

In [3]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS')
player_ranking_steam = Projection(path_data=path_data,year=year,model='Steamer')

In [476]:
class Draft:

    def __init__(self, projections_object, 
                 draft_position = 6,
                 number_teams = 12,
                 roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5},
                 batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS'],
                 pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] ):

        self.number_rounds = sum(roster_spots.values())
        self.draft_position = draft_position - 1 # e.g., 1st pick is 0!
        self.round_number = 0
        self.draft_number = 0
        self.player_projections = projections_object
        self.remaining_ranked_players = projections_object.all_rank 
        #self.roto_standings = []
        self.roto_stats_batting = pd.DataFrame(columns =  batter_stats[1:])
        self.roto_stats_pitching = pd.DataFrame(columns =  pitcher_stats[1:])
        self.teams = {}
        for i in np.arange(number_teams):
            roto_stats = {}
            roto_stats['batting_stats'] = pd.DataFrame(columns =  batter_stats)
            roto_stats['pitching_stats'] = pd.DataFrame(columns =  pitcher_stats)
            roto_stats['roster_spots'] = roster_spots.copy()
            roto_stats['roster'] = {}
            self.teams[i] = roto_stats
    
    # Find Resulting Standings
    def tabulate_roto(self, teams):
        roto_stats_batting = self.roto_stats_batting.copy()
        roto_stats_pitching = self.roto_stats_pitching.copy()
        for iteam in np.arange(len(teams)):
            raw_team_batting = teams[iteam]['batting_stats']
            raw_team_pitching = teams[iteam]['pitching_stats']
            roto_team_batting = raw_team_batting.mean()
            roto_team_pitching = raw_team_pitching.mean()
            if 'AVG' in raw_team_batting:
                roto_team_batting['AVG'] = (raw_team_batting['AVG']*raw_team_batting['AB']).sum()/raw_team_batting['AB'].sum()
            if 'OPS' in raw_team_batting:
                roto_team_batting['OPS'] = (raw_team_batting['OPS']*raw_team_batting['AB']).sum()/raw_team_batting['AB'].sum() 
            if 'ERA' in raw_team_pitching:
                roto_team_batting['ERA'] = (raw_team_pitching['ERA']*raw_team_pitching['IP']).sum()/raw_team_pitching['IP'].sum() 
            if 'WHIP' in raw_team_pitching:
                roto_team_batting['WHIP'] = (raw_team_pitching['WHIP']*raw_team_pitching['IP']).sum()/raw_team_pitching['IP'].sum()
            batting_stat_names = self.roto_stats_batting.columns.values.tolist()
            pitching_stat_names = self.roto_stats_pitching.columns.values.tolist()
            roto_stats_batting = roto_stats_batting.append(roto_team_batting[batting_stat_names],ignore_index = True)
            roto_stats_pitching = roto_stats_pitching.append(roto_team_pitching[pitching_stat_names],ignore_index = True)
            
        roto_team_stats = pd.concat([roto_stats_batting, roto_stats_pitching], axis=1, sort=False)
        roto_standings_avgs = pd.concat([roto_stats_batting.rank(ascending=False), roto_stats_pitching.rank(ascending=False).rename(columns={"BB": "BBP"})], axis=1, sort=False)
        roto_standings_cnts = pd.concat([roto_stats_batting.rank(), roto_stats_pitching.rank().rename(columns={"BB": "BBP"})], axis=1, sort=False)
        avg_stats = ['CS','BBP','ERA','WHIP','BSV']
        for avg_stat in avg_stats:
            if avg_stat in roto_standings_avgs:
                roto_standings_cnts[avg_stat] = roto_standings_avgs[avg_stat]
        roto_standings = roto_standings_cnts.sum(axis=1).sort_values(ascending=False)
        roto_placement = roto_standings.index.get_loc(self.draft_position)
        #pdb.set_trace()
        return roto_stats_batting, roto_stats_pitching, roto_standings, roto_placement
    
    
    # Do the entire draft one round at a time
    def draft_all(self):
        for iround in np.arange(self.number_rounds):
            self.draft_round(iround)
            self.round_number += 1
            #pdb.set_trace()
        self.roto_stats_batting, self.roto_stats_pitching, self.roto_standings, self.roto_placement = self.tabulate_roto(self.teams)
      
    
    def draft_round(self, round_key):
        
        # Reverse draft order every other round
        draft_order = np.arange(len(self.teams))
        if round_key % 2 == 1:
            draft_order = draft_order[::-1]
        
        # Reset draft_number at start of round
        self.draft_number = len(self.teams) - len(draft_order)
        
        # Draft each round one team at a time
        for iteam in draft_order:
            
            # When team is draft_position, search for best pick.
            #if iteam == self.draft_position:
            #    best_pick = self.find_best_pick(iteam)
            #    self.teams, self.remaining_ranked_players = self.draft_next_best(iteam, force_pick = best_pick)
            #else: 
            #    self.teams, self.remaining_ranked_players = self.draft_next_best(iteam)
            self.teams, self.remaining_ranked_players = self.draft_next_best(iteam)
            self.draft_number += 1
            
        #pdb.set_trace()

        
    def draft_remaining(self, teams, df):
        for iround in range(self.round_number,self.number_rounds):
            #self.draft_round(iround)
            
            draft_order = np.arange(len(self.teams))
            # Reverse draft order every other round
            if iround % 2 == 1:
                draft_order = draft_order[::-1]
            
            if iround == self.round_number:
                draft_order = draft_order[self.draft_position + 1:] # wont work if draw first or last picks...
                
            for iteam in draft_order:
                teams, df = self.draft_next_best_debug(iteam, teams = teams, df = df)

        #pdb.set_trace()
        return teams
        
        
    def find_best_pick(self, team_key):
        # find_best_pick returns iloc, the index (of df) of the optimal pick
        
        df = self.remaining_ranked_players
        all_unfilled_positions = [k for (k,v) in self.teams[team_key]['roster_spots'].items() if v > 0]
        # Figure out if only BN is left, in which case all options are open
        if ('BN' in all_unfilled_positions) and (len(all_unfilled_positions) == 1):
            unfilled_positions = ['C', '1B', '2B', '3B', 'SS', 'OF','Util','SP','RP']
        else: 
            # If UTIL is unfilled, all fielding positions are options.  
            if self.teams[team_key]['roster_spots']['UTIL'] > 0:
                #ups = 'C|1B|2B|3B|SS|OF'
                unfilled_positions = ['C', '1B', '2B', '3B', 'SS', 'OF']
                if self.teams[team_key]['roster_spots']['SP'] > 0:
                    #ups = ups+'|SP'
                    unfilled_positions.append('SP')
                if self.teams[team_key]['roster_spots']['RP'] > 0:
                    #ups = ups+'|RP'
                    unfilled_positions.append('RP')
                #unfilled_positions = ups.split('|')
            else: 
                unfilled_positions = [k for (k,v) in self.teams[team_key]['roster_spots'].items() if v > 0]
                #ups = '|'.join(unfilled_positions)
        ups = '|'.join(unfilled_positions)
        print(ups)  
        
        # Begin with next best player.        
        idx_eligible = [0]
        
        # Find index of best player at each remaining position
        for iunfilled in unfilled_positions:
            # WANT TO PUT GOOD PLAYERS IN UTIL, BUT NOT BN            
            if (iunfilled != 'UTIL') & (iunfilled != 'BN'):
                idx_position = [i for i, val in enumerate(df.EligiblePosition.str.contains(iunfilled)) if val] 
                filled = False
                jdx = 0
                while filled == False:
                    if idx_position[jdx] in idx_eligible:
                        jdx+=1
                    else:
                        idx_eligible.append(idx_position[jdx])
                        filled = True
        
        # Get rid of doubles (1B and OF is particularly prone)
        idx_eligible = np.unique(idx_eligible)
        print('Picking from:')
        print(df.iloc[idx_eligible])
        #pdb.set_trace()
        
        #################################
        # START OF LOOP TO FIND BEST PLAYER
        player_based_draft_outcomes = {}

        # Loop over eligible players, then finish the draft
        for iposition, icounter in zip(idx_eligible, range(len(idx_eligible))):
            
            # make a copy of teams to finish drafting
            teams_copy = self.teams.copy()
            df_copy = df.copy()

            # Draft looping through idx_eligible
            df_copy,drafted_player=df_copy.drop(df_copy.iloc[iposition:iposition+1].index),df_copy.iloc[iposition:iposition+1]
            
            # Already know there are open positions, so is this necessary?
            eligible_positions = drafted_player.EligiblePosition.values[0] 
            position = self.get_optimal_position(eligible_positions, teams_copy[team_key]['roster_spots'])
            
            # Check if Position is a Pitcher or Batter
            if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
                teams_copy[team_key]['pitching_stats'] = teams_copy[team_key]['pitching_stats'].append(statline[0:1])

            else:
                idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
                teams_copy[team_key]['batting_stats'] = teams_copy[team_key]['batting_stats'].append(statline[0:1])

            # LOOP OVER WHOLE REST OF THE DRAFT HERE...
            # pdb.set_trace()
            teams_copy = self.draft_remaining(teams_copy, df_copy)
            
            # Calculate the best pseudo-standings
            pseudo_standings = self.tabulate_roto(teams_copy)
            
            #pdb.set_trace()
            # Decide best choice
            # LOGIC...
            
            # Store the result.  
            #player_based_draft_outcomes = {idx_player:}
            
        # Decide on best choice and return 
        best_pick = 5
        
        return best_pick
        # END OF LOOP TO FIND BEST PLAYER
        #################################
    
    # Strategy is to take the best possible player, even if that means putting them in UTIL or BN (maybe BN should reconsidered...)
    def draft_next_best(self, team_key, teams = 'teams', df = pd.DataFrame(), force_pick = False):
                
        if bool(teams == 'teams'):
            teams = self.teams
        
        if df.empty == True:
            df = self.remaining_ranked_players

        if (force_pick == False):
            pick_ok = False
            idf = 0
            while pick_ok == False:
                df_copy = df.copy()
                pick_ok = True
                #print('Round '+str(self.round_number))

                df_copy,drafted_player=df_copy.drop(df_copy.iloc[idf:idf+1].index),df_copy.iloc[idf:idf+1]
                eligible_positions = drafted_player.EligiblePosition.values[0]

                if eligible_positions == 'Util':
                    open_positions = [teams[team_key]['roster_spots']['UTIL']]
                else:
                    open_positions = [teams[team_key]['roster_spots'][i] for i in eligible_positions.split('/')]
                
                # If there are no SP, RP, or Fielding Positions left, try P, UTIL, or BN
                if np.sum(open_positions) == 0:
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        if teams[team_key]['roster_spots']['P'] > 0:
                            position = 'P'
                            df = df_copy
                        elif teams[team_key]['roster_spots']['BN'] > 0:
                            position = 'BN'
                            df = df_copy
                        else:
                            position = 'No Roster Spots Left'
                            pick_ok = False
                            
                    else:
                        if teams[team_key]['roster_spots']['UTIL'] > 0:
                            position = 'UTIL'
                            df = df_copy
                        elif teams[team_key]['roster_spots']['BN'] > 0:
                            position = 'BN'
                            df = df_copy
                        else:
                            position = 'No Roster Spots Left'
                            pick_ok = False

                else:

                    df = df_copy                            
                    position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
                    #print(drafted_player.PLAYER+' is a '+position)
                    pick_ok = True

                if pick_ok == True:
                    # Check if Position is a Pitcher or Batter
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
                        teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

                    else:
                        idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
                        teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])
                        #pdb.set_trace()
                    #print('Team '+ str(team_key) +' Drafting '+drafted_player.iloc[0].PLAYER+' for '+position)
                else:
                    #print('Not Drafting '+drafted_player.PLAYER+' for '+position)
                    # Try while loop again after incrementing idf
                    idf += 1

        else:
            #pdb.set_trace()
            pick = force_pick -1 
            df,drafted_player=df.drop(df.iloc[pick:pick+1].index),df.iloc[pick:pick+1]
            eligible_positions = drafted_player.EligiblePosition.values[0]
            position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
            
            if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.pitchers_stats[idx_player][teams[team_key]['pitching_stats'].keys()]
                teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

            else:
                idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.hitters_stats[idx_player][teams[team_key]['batting_stats'].keys()]
                teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])        
                
            print('Team '+ str(team_key) +' FORCED to pick '+drafted_player.iloc[0].PLAYER+' for '+position)
        
        # Eliminate one roster_spot
        teams[team_key]['roster_spots'][position] -= 1

        # Add Player to Roster
        if position in self.teams[team_key]['roster']:
            teams[team_key]['roster'][position] = [teams[team_key]['roster'][position], drafted_player.PLAYER.values[0]]
        else:
            teams[team_key]['roster'][position] = drafted_player.PLAYER.values[0]
            
        return teams, df

    # Strategy is to take the best possible player, even if that means putting them in UTIL or BN (maybe BN should reconsidered...)
    def draft_next_best_debug(self, team_key, teams = 'teams', df = pd.DataFrame(), force_pick = False):
        
        #pdb.set_trace()
        
        if bool(teams == 'teams'):
            teams = self.teams
        
        if df.empty == True:
            df = self.remaining_ranked_players

        if (force_pick == False):
            pick_ok = False
            idf = 0
            while pick_ok == False:
                df_copy = df.copy()
                pick_ok = True
                #print('Round '+str(self.round_number))

                # Figure out what positions are still unfilled.
                #unfilled_positions = [k for (k,v) in teams[team_key]['roster_spots'].items() if v > 0]
                #unfilldpns = '|'.join(unfilled_positions)
                #print(unfilldpns)

                # Find indices of all remaining players in positions of need.
                #idx_eligible = [i for i, val in enumerate(df_copy.EligiblePosition.str.contains(unfilldpns)) if val]

                df_copy,drafted_player=df_copy.drop(df_copy.iloc[idf:idf+1].index),df_copy.iloc[idf:idf+1]
                if drafted_player.iloc[0].PLAYER == "Yonder Alonso":
                    pdb.set_trace()
                eligible_positions = drafted_player.EligiblePosition.values[0]

                # If they are Util, they are only UTIL
                if eligible_positions == 'Util':
                    open_positions = [teams[team_key]['roster_spots']['UTIL']]
                else:
                    open_positions = [teams[team_key]['roster_spots'][i] for i in eligible_positions.split('/')]
                
                #print(np.asarray([*teams[team_key]['roster_spots']])[open_positions])
                #pdb.set_trace()
                # If there are no SP, RP, or Fielding Positions left, try P, UTIL, or BN
                if np.sum(open_positions) == 0:
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        if teams[team_key]['roster_spots']['P'] > 0:
                            position = 'P'
                            df = df_copy
                        elif teams[team_key]['roster_spots']['BN'] > 0:
                            position = 'BN'
                            df = df_copy
                        else:
                            position = 'No Roster Spots Left'
                            pick_ok = False
                            
                    else:
                        if teams[team_key]['roster_spots']['UTIL'] > 0:
                            position = 'UTIL'
                            df = df_copy
                        elif teams[team_key]['roster_spots']['BN'] > 0:
                            position = 'BN'
                            df = df_copy
                        else:
                            position = 'No Roster Spots Left'
                            pick_ok = False

                else:

                    df = df_copy                            
                    position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
                    #print(drafted_player.PLAYER+' is a '+position)
                    pick_ok = True
                
                #pdb.set_trace()

                if pick_ok == True:
                    # Check if Position is a Pitcher or Batter
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
                        teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

                    else:
                        idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
                        teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])
                        #pdb.set_trace()
                    #print('Team '+ str(team_key) +' Drafting '+drafted_player.iloc[0].PLAYER+' for '+position)
                else:
                    #print('Not Drafting '+drafted_player.PLAYER+' for '+position)
                    # Try while loop again after incrementing idf
                    print(idf)
                    if idf > 400:
                        pdb.set_trace()
                    idf += 1

        else:
            #pdb.set_trace()
            pick = force_pick -1 
            df,drafted_player=df.drop(df.iloc[pick:pick+1].index),df.iloc[pick:pick+1]
            eligible_positions = drafted_player.EligiblePosition.values[0]
            position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
            
            if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.pitchers_stats[idx_player][teams[team_key]['pitching_stats'].keys()]
                teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

            else:
                idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.hitters_stats[idx_player][teams[team_key]['batting_stats'].keys()]
                teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])        
                
            print('Team '+ str(team_key) +' FORCED to pick '+drafted_player.iloc[0].PLAYER+' for '+position)
        
        # Eliminate one roster_spot
        teams[team_key]['roster_spots'][position] -= 1

        # Add Player to Roster
        if position in self.teams[team_key]['roster']:
            teams[team_key]['roster'][position] = [teams[team_key]['roster'][position], drafted_player.PLAYER.values[0]]
        else:
            teams[team_key]['roster'][position] = drafted_player.PLAYER.values[0]
            
        #pdb.set_trace()
        return teams, df
        
    def get_optimal_position(self, positions_in, roster_spots):
        # Return in the order of most valuable position to fill
        for single_position in positions_in.split('/'):
            #pdb.set_trace()
            if (single_position == 'C') & (roster_spots['C'] > 0):
                return  'C'
            elif (single_position == '2B') & (roster_spots['2B'] > 0):
                return  '2B'
            elif (single_position == 'SS') & (roster_spots['SS'] > 0):
                return  'SS'
            elif (single_position == 'OF') & (roster_spots['OF'] > 0):
                return  'OF'
            elif (single_position == '3B') & (roster_spots['3B'] > 0):
                return  '3B'
            elif (single_position == '1B') & (roster_spots['1B'] > 0):
                return  '1B'
            elif (single_position == 'Util') & (roster_spots['UTIL'] > 0):
                return 'UTIL'
            else:
                return  single_position
                    

In [477]:
zips = Draft(player_ranking_zips, number_teams = 12)
steam = Draft(player_ranking_steam, number_teams = 12)

In [478]:
zips.draft_all()

In [482]:
zips.teams[4]

{'batting_stats':         AB    R   1B  2B 3B  HR  RBI  SB   BB    AVG    OPS
 20123  559  115   91  33  5  36  126  11  113  0.295  0.978
 19556  495   85   76  30  1  35  107   4   66  0.287  0.933
 4949   494   86   59  24  1  43  116   2   63  0.257  0.915
 14162  406   63   63  23  1  22   85   3   53  0.268  0.845
 10071  569   82  103  25  3  17   57  41   57  0.260  0.733
 14225  508   69   81  26  2  24   84  16   44  0.262  0.787
 3516   595   73  105  27  2  22   90   3   50  0.262  0.745
 15161  334   55   46  18  1  18   61   0   40  0.249  0.803,
 'pitching_stats':         IP   W  L CG SHO  SV  BB   SO   ERA  WHIP BSV
 5    174.0  13  6  1   1   0  39  236  3.00  0.98   0
 227   74.3   6  3  0   0   0  18   88  3.15  1.05   0
 538   63.0   6  4  0   0   0  33   83  3.71  1.29   0
 296   60.0   4  2  0   0  29  15   78  3.15  1.03   2
 558   62.3   3  2  0   0   0  21   67  4.04  1.25   0
 30   144.7  11  7  1   1   0  38  145  3.98  1.20   0
 541   42.3   3  2  0   0  17 

In [7]:
steam.draft_all()

In [8]:
steam.roto_placement

9

In [8]:
zips.roto_standings

0     171.0
9     163.0
7     156.0
1     144.5
10    143.0
4     123.0
6     117.0
11    114.5
3     114.0
8     108.0
2     103.5
5     102.5
dtype: float64

In [9]:
steam.roto_standings

0     180.0
9     162.0
10    153.0
11    140.5
1     130.0
4     125.0
7     122.0
3     122.0
2     116.5
6     115.0
5     114.0
8      80.0
dtype: float64

In [10]:
steam.teams[6]['roster']

{'1B': 'Cody Bellinger',
 'SS': 'Fernando Tatis',
 '2B': 'Ozzie Albies',
 'RP': ['Aroldis Chapman', 'Tyler Duffey'],
 'UTIL': 'Carlos Santana',
 'SP': ['Drew Pomeranz', 'Brandon Woodruff'],
 'P': [['Zack Britton', 'John Brebbia'], 'Colin Poche'],
 'BN': [[[['Yoan López', 'Keynan Middleton'], 'Mark Melancon'], 'Luis Patiño'],
  'Tyler Rogers'],
 '3B': 'Luis Arraez',
 'OF': [['Nick Senzel', 'Willie Calhoun'], 'Franmil Reyes'],
 'C': 'Carson Kelly'}